<a href="https://colab.research.google.com/github/Mona1811k/Anomaly-Detection-in-CCTV-Footage-using-Deep-Learning-and-with-Alerting-Sytsem/blob/main/smart_ser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output,Video
import keras
import numpy as np
import cv2

In [ ]:
scvd_train_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train'
scvd_test_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test'
scvd_classes = ['Normal', 'Violence','Weaponized']

In [ ]:
# Define the base directory and categories
base_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train'
categories = ['Normal', 'Violence', 'Weaponized']

In [ ]:
import os

# Function to count videos in each category
def count_videos(base_dir, categories):
    video_count = {}

    for category in categories:
        category_path = os.path.join(base_dir, category)

        if not os.path.exists(category_path):
            print(f"Directory for category '{category}' not found.")
            video_count[category] = 0
            continue

        # List all video files in the category directory
        video_files = [f for f in os.listdir(category_path) if f.endswith(('.mp4', '.avi', '.mkv'))]
        video_count[category] = len(video_files)

    return video_count

# Call the function and print the counts
video_counts = count_videos(base_dir, categories)

print("Video counts per category:")
for category, count in video_counts.items():
    print(f"{category}: {count} videos")


Video counts per category:
Normal: 872 videos
Violence: 970 videos
Weaponized: 832 videos
